<div dir="rtl">

# חלק ראשון: POST

<div dir="rtl">

# 1.האם מספר חשבון 771641 רכש מניות של אפל? (שם המניה הוא aapl)

In [18]:
import pandas as pd
import requests
import json

In [29]:
query = {
    'query' : {
        "account_id": 771641
    }
}

In [30]:
stock_payload = json.dumps(stock_query)
my_header = {'token' : 'super_secret_token'}

In [32]:
stocks = requests.post('https://mongo-rest-week8.herokuapp.com/stocks', data=stock_payload, headers=my_header)
stocks.text

'{"documents":[{"account_id":771641,"transaction_count":15,"bucket_start_date":"1991-01-01T00:00:00","bucket_end_date":"2016-12-16T00:00:00","transactions":[{"date":"2012-03-14T00:00:00","amount":2931,"transaction_code":"buy","symbol":"sap","price":"65.0757645452902835359054733999073505401611328125","total":"190737.0658822458210437389425"},{"date":"2000-09-01T00:00:00","amount":8077,"transaction_code":"sell","symbol":"nvda","price":"12.417421460358962548298222827725112438201904296875","total":"100295.5131353193405026047458"},{"date":"2015-03-17T00:00:00","amount":5935,"transaction_code":"buy","symbol":"goog","price":"548.2270901499422279812279157340526580810546875","total":"3253727.780039907123068587680"},{"date":"2013-08-12T00:00:00","amount":40,"transaction_code":"buy","symbol":"adbe","price":"47.29094385057584304377087391912937164306640625","total":"1891.637754023033721750834957"},{"date":"2013-11-22T00:00:00","amount":9689,"transaction_code":"buy","symbol":"aapl","price":"68.949484

In [42]:
stock_documents = json.loads(stocks.text)['documents'][0]
transactions = stock_documents['transactions']

In [43]:
transactions = pd.DataFrame(transactions)
transactions.head()

,date,amount,transaction_code,symbol,price,total
0,2012-03-14T00:00:00,2931,buy,sap,65.0757645452902835359054733999073505401611328125,190737.0658822458210437389425
1,2000-09-01T00:00:00,8077,sell,nvda,12.4174214603589625482982228277251124382019042...,100295.5131353193405026047458
2,2015-03-17T00:00:00,5935,buy,goog,548.2270901499422279812279157340526580810546875,3253727.780039907123068587680
3,2013-08-12T00:00:00,40,buy,adbe,47.29094385057584304377087391912937164306640625,1891.637754023033721750834957
4,2013-11-22T00:00:00,9689,buy,aapl,68.9494847512784048149114823900163173675537109375,668051.5577551364642516773529


In [44]:
casting = {
    'amount': 'int'
    , 'price': 'float'
    , 'total': 'float'
}

transactions = df.astype(casting)
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              15 non-null     object 
 1   amount            15 non-null     int32  
 2   transaction_code  15 non-null     object 
 3   symbol            15 non-null     object 
 4   price             15 non-null     float64
 5   total             15 non-null     float64
dtypes: float64(2), int32(1), object(3)
memory usage: 788.0+ bytes


In [49]:
bought_apple = (transactions['symbol']  == 'aapl') & (transactions['transaction_code'] == 'buy')
transactions[bought_apple].size > 0

True

<div dir="rtl">

# 2. מהו סכום הרכישה הממוצע של החשבון?

In [55]:
mean_total_amount = transactions.groupby('transaction_code')['total'].mean()
mean_total_amount['buy']

1476763.455220636

<div dir="rtl">

# 3. מהו סכום המכירה הממוצע של החשבון?

In [56]:
mean_total_amount['sell']

834498.4161406945

<div dir="rtl">

# 4. מהו שם האדם שבבעלותו החשבון? האם יש לו יותר מחשבון אחד?

In [61]:
investor_query = {
    'query': {
        "accounts": 771641
    }
}

In [62]:
investor_payload = json.dumps(investor_query)

In [64]:
investor = requests.post('https://mongo-rest-week8.herokuapp.com/investors', data=investor_payload, headers=my_header)
investor.text

'{"documents":[{"username":"hmyers","name":"Dana Clarke","address":"50047 Smith Point Suite 162\\nWilkinsstad, PA 04106","birthdate":"1969-06-21T02:39:20","email":"vcarter@hotmail.com","accounts":[627629,55958,771641],"tier_and_details":{"4c207e65857742f89d8155139b24c0f0":{"tier":"Silver","benefits":["car rental insurance","travel insurance"],"active":true,"id":"4c207e65857742f89d8155139b24c0f0"},"c04ee1d7093449148a3cc3bbca398529":{"tier":"Platinum","benefits":["24 hour dedicated line","dedicated account representative"],"active":true,"id":"c04ee1d7093449148a3cc3bbca398529"},"1e64a51089c54d08911baf77be6b3713":{"tier":"Gold","benefits":["concert tickets","dedicated account representative"],"active":true,"id":"1e64a51089c54d08911baf77be6b3713"}}}]}'

In [70]:
investor_documents = json.loads(investor.text)['documents'][0]
print("Investor's name is:", investor_documents['name'])

Investor's name is: Dana Clarke


In [77]:
accounts_count = len(investor_documents['accounts'])
print('Investor has', accounts_count, 'accounts')

Investor has 3 accounts


<div dir="rtl">

# 5. שאלת אתגר: עכשיו כשיש ברשותכם את כל החשבונות של המשקיע/ה, חשבו מחדש את סכום הקניה הממוצע ואת סכום הרכישה הממוצע.
שימו לב לחשב אותו לפי היסטוריית הפעולות של כל החשבונות.

**רמז**: הסינטקס לשליפת in בapi הזה הוא: 

`field' : {'$in' : [value1, value2, value3]}'`

In [82]:
investor_accounts = investor_documents['accounts']

full_stock_query = {
    'query' : {
        "account_id": {'$in': investor_accounts}
    }
}

In [85]:
full_stock_payload = json.dumps(full_stock_query)

In [86]:
full_stocks = requests.post('https://mongo-rest-week8.herokuapp.com/stocks', data=full_stock_payload, headers=my_header)
full_stocks.text

'{"documents":[{"account_id":771641,"transaction_count":15,"bucket_start_date":"1991-01-01T00:00:00","bucket_end_date":"2016-12-16T00:00:00","transactions":[{"date":"2012-03-14T00:00:00","amount":2931,"transaction_code":"buy","symbol":"sap","price":"65.0757645452902835359054733999073505401611328125","total":"190737.0658822458210437389425"},{"date":"2000-09-01T00:00:00","amount":8077,"transaction_code":"sell","symbol":"nvda","price":"12.417421460358962548298222827725112438201904296875","total":"100295.5131353193405026047458"},{"date":"2015-03-17T00:00:00","amount":5935,"transaction_code":"buy","symbol":"goog","price":"548.2270901499422279812279157340526580810546875","total":"3253727.780039907123068587680"},{"date":"2013-08-12T00:00:00","amount":40,"transaction_code":"buy","symbol":"adbe","price":"47.29094385057584304377087391912937164306640625","total":"1891.637754023033721750834957"},{"date":"2013-11-22T00:00:00","amount":9689,"transaction_code":"buy","symbol":"aapl","price":"68.949484

In [115]:
full_stock_documents = json.loads(full_stocks.text)['documents']
full_transactions = pd.DataFrame()
for account_data in full_stock_documents:
    account_df = pd.DataFrame(account_data['transactions'])
    account_df['account_id'] = account_data['account_id']
    full_transactions = full_stock_df.append(account_df)

In [116]:
full_transactions.head()

,date,amount,transaction_code,symbol,price,total,account_id
0,2012-03-14T00:00:00,2931,buy,sap,65.0757645452902835359054733999073505401611328125,190737.0658822458210437389425,771641
1,2000-09-01T00:00:00,8077,sell,nvda,12.4174214603589625482982228277251124382019042...,100295.5131353193405026047458,771641
2,2015-03-17T00:00:00,5935,buy,goog,548.2270901499422279812279157340526580810546875,3253727.780039907123068587680,771641
3,2013-08-12T00:00:00,40,buy,adbe,47.29094385057584304377087391912937164306640625,1891.637754023033721750834957,771641
4,2013-11-22T00:00:00,9689,buy,aapl,68.9494847512784048149114823900163173675537109375,668051.5577551364642516773529,771641


In [117]:
full_transactions = full_transactions.astype(casting)
full_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 89
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              205 non-null    object 
 1   amount            205 non-null    int32  
 2   transaction_code  205 non-null    object 
 3   symbol            205 non-null    object 
 4   price             205 non-null    float64
 5   total             205 non-null    float64
 6   account_id        205 non-null    int64  
dtypes: float64(2), int32(1), int64(1), object(3)
memory usage: 12.0+ KB


In [118]:
mean_total_amount = full_transactions.groupby('transaction_code')['total'].mean()
mean_total_amount['buy']

797568.049568806

In [119]:
mean_total_amount['sell']

800996.0820607743

 <div dir="rtl">

# חלק שני: GET

<div dir="rtl">

# 1. לרשותכם API ציטוטים מעוררי השראה. בצעו בקשה וקבלו JSON עם משפט לחיים.

`https://zenquotes.io/api/random/`

In [125]:
response = requests.get('https://zenquotes.io/api/random/')
json.loads(response.text)[0]['q']

'You talk when you cease to be at peace with your thoughts.'